## Домашняя работа
ФИО: Мансурова А.Р.

## Задание 1
Реализуйте метакласс ThreadSafeSingleton, который обеспечивает создание только одного экземпляра класса, даже в многопоточной среде.
Используйте `from threading import Lock`


### проверка задания 1

In [ ]:
from threading import Lock

class ThreadSafeSingleton(type):
    _instances = {}
    _lock = Lock()

    def __call__(cls, *args, **kwargs):
        with cls._lock:
            if cls not in cls._instances:
                cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]


class DatabasePool(metaclass=ThreadSafeSingleton):
    def __init__(self):
        if not hasattr(self, '_connections'):
            self._connections = ["conn_1", "conn_2", "conn_3"]
            self._index = 0

    def get_connection(self):
        conn = self._connections[self._index]
        self._index = (self._index + 1) % len(self._connections)
        return conn


# Создайте 3 экземпляра DatabasePool
pool1 = DatabasePool()
pool2 = DatabasePool()
pool3 = DatabasePool()

# Убедитесь, что это один и тот же объект
assert pool1 is pool2 is pool3

# Проверьте, что соединения разделяются между экземплярами
conn1 = pool1.get_connection()
conn2 = pool2.get_connection()
assert conn1 != conn2

## Задание 2

Создайте метакласс, который считает, сколько раз создавался каждый класс.

Требования:
1. Метакласс должен иметь атрибут _counters
1. При создании экземпляра класса счетчик должен увеличиваться
1. Добавьте метод get_count(), который возвращает количество созданных экземпляров

### проверка задания 2

In [ ]:
class CountInstancesMeta(type):
    _counters = {}

    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)
        mcs._counters[cls] = 0
        cls.get_count = classmethod(lambda cls_inner: mcs._counters[cls_inner])
        return cls

    def __call__(cls, *args, **kwargs):
        CountInstancesMeta._counters[cls] += 1
        return super().__call__(*args, **kwargs)


class User(metaclass=CountInstancesMeta):
    def __init__(self, name):
        self.name = name


class Product(metaclass=CountInstancesMeta):
    def __init__(self, name):
        self.name = name


# Проверка
user1 = User("Alice")
user2 = User("Bob")
product1 = Product("Laptop")

print(User.get_count())    # Должно быть 2
print(Product.get_count()) # Должно быть 1

2
1


## Задание 3

Создайте метакласс, который автоматически добавляет метод describe() в каждый класс.

Требования:
1. Метод describe() должен возвращать строку с именем класса
1. Используйте метакласс для создания классов Car и Book

### проверка задания 3

In [8]:
class DescribeMeta(type):
    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)

        def describe(self):
            return f"Это объект класса {self.__class__.__name__}"

        cls.describe = describe
        return cls


class Car(metaclass=DescribeMeta):
    def __init__(self, brand):
        self.brand = brand


class Book(metaclass=DescribeMeta):
    def __init__(self, title):
        self.title = title


car = Car("Toyota")
book = Book("Python для начинающих")

print(car.describe())  # Это объект класса Car
print(book.describe()) # Это объект класса Book

Это объект класса Car
Это объект класса Book


## Задание 4

Создайте метакласс, который проверяет, что у класса есть метод save(). Можно использовать `__new__`

Требования:
1. Если у класса нет метода save(), метакласс должен выдать ошибку
1. Создайте класс User с методом save()
1. Попробуйте создать класс Message без метода save() (должна быть ошибка)

### проверка задания 4

In [ ]:
class SaveMeta(type):
    def __new__(mcs, name, bases, namespace):
        if 'save' not in namespace:
            raise TypeError(f"Класс '{name}' должен содержать метод 'save()'")
        if not callable(namespace['save']):
            raise TypeError(f"Атрибут 'save' в классе '{name}' должен быть методом")
        
        return super().__new__(mcs, name, bases, namespace)


class User(metaclass=SaveMeta):
    def __init__(self, name):
        self.name = name

    def save(self):
        print(f"Пользователь {self.name} сохранён")


# Этот код вызовет ошибку (раскомментируй, чтобы проверить):
class Message(metaclass=SaveMeta):
     def __init__(self, text):
        self.text = text


# Проверка
user = User("Alice")
user.save()  # Должно работать → выведет: "Пользователь Alice сохранён"

TypeError: Класс 'Message' должен содержать метод 'save()'